<h1>Coursera Capstone Project</h1>

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


<h2>1. Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto</h2>

1.1 
<br>Scrap the postal code table from the wiki website
<br>Use BeautifulSoup to read the table content

In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_html_raw = requests.get(wiki_url).text
soup = BeautifulSoup(wiki_html_raw, "lxml")
wiki_html_table = soup.find("table", attrs={"class": "wikitable"})
wiki_html_table_data = wiki_html_table.tbody.find_all("tr")
postal_code = []
borough = []
neighbourhood = []

for i in range(len(wiki_html_table_data)):
    if i != 0:
        cols = wiki_html_table_data[i].find_all("td")
        postal_code.append(cols[0].text.replace("\n",""))
        borough.append(cols[1].text.replace("\n",""))
        neighbourhood.append(cols[2].text.replace("\n",""))

        data = pd.DataFrame()
data['PostalCode'] = postal_code
data['Borough'] = borough
data['Neighbourhood'] = neighbourhood

data.head()

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


1.2 
<br>Process the table

In [4]:
# Ignore cells with a borough that is Not assigned
data = data[data['Borough'] != 'Not assigned']

# Find duplicate Postal Code
data[data["PostalCode"].duplicated()]

# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
for i in range(len(data)):
    if data.iloc[i]['Neighbourhood'] == 'Not assigned':
        data.iloc[i]['Neighbourhood'] = data.iloc[i]['Borough']

data.head()

,PostalCode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
data.shape

(103, 3)

1.3 
<br>Get geospatial data
<br>Merge the data with geospatial data

In [6]:
geo_data = pd.read_csv("http://cocl.us/Geospatial_data")
data = data.join(geo_data.set_index('Postal Code'), on='PostalCode')
data.head()

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


1.4 
<br>Generate a map to visualize the result

In [13]:
#!conda install -c conda-forge folium=0.5.0 --yes
import folium

In [16]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[43.6, -79.3], zoom_start=10, tiles='StamenToner')

# add markers to map
for lat, lng, borough, neighborhood in zip(data['Latitude'], data['Longitude'], data['Borough'], data['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#3F5161',
        fill=True,
        fill_color='#427CAC',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto